<h1><center>cs1001.py , Tel Aviv University, Fall 2017-2018</center></h1>
<img src="http://www.pngall.com/wp-content/uploads/2016/05/Python-Logo-PNG-Image-180x180.png" width=50/>

# Recitation 8

We have reviewed some properties of prime numbers and used them for primality testing. We revied the Diffie-Hellman protocol for finding a shared secret key and also tried to crack it. At the second part of the recitation, we discussed OOP and special methods that allow us to define operators in Python.

#### Takeaways:
- The probabilistic function is_prime, that uses Fermat's primality test, can be used to detect primes quickly and efficiently, but has a (very small) probability of error. Its time complexity is $O(n^3)$, where $n$ is the number of bits of the input.
- The DH protocol relies on two main principles: the following equality $(g^{a}\mod p)^b \mod p = g^{ab} \mod p $ and the (believed) hardness of the discrete log problem (given $g,p$, and $x = g^{a} \mod p$ finding $a$ is hard). Make sure you understand the details of the protocol.
- OOP allows us to create classes at will and to define complex objects. Remember that the most important thing is to find a good inner representation of the object so that you will have an easy time working with it.

#### Code for printing several outputs in one cell (not part of the recitation):

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Primes and Diffie-Hellman

#### Primality test using Ferma's witness

Fermat's little theorem: if $p$ is prime and $1 < a < p$, then $a^{p-1} (\textrm{mod}\ p) \equiv 1$

Equivalently: if $m$ is not a prime then there exists $1 < a < m$ such that $a^{m-1} (\textrm{mod}\ m) \not\equiv 1$. Such a number $a$ is called a witness to the fact that $m$ is not prime.

We can use Fermat's little theorem in order test whether a given number is prime. Note that if the number has $n$ bits than testing all possible $a$-s will require $O(2^n)$ iterations (a lot!).

Instead, we will try 100 random $a$-s in the range and see if one works as a witness.

In [4]:
import random

def is_prime(m, show_witness=False):

    """ probabilistic test for m's compositeness """

    for i in range(0,100):
        a = random.randint(1,m-1) # a is a random integer in [1..m-1]
        if pow(a,m-1,m) != 1:
            if show_witness:  # caller wishes to see a witness
                print(m,"is composite","\n",a,"is a witness, i=",i+1)
            return False

    return True

For $a,b,c$ of at most $n$ bits each, time complexity of modpower is $O(n^3)$

In [ ]:
def modpower(a, b, c):
    """ computes a**b modulo c, using iterated squaring """
    result = 1
    while b>0: # while b is nonzero
        if b%2 == 1: # b is odd
            result = (result * a) % c
        a = (a*a) % c
        b = b//2
    return result

#### The probability for error:
First, notice that if the function says that an imput number $m$ is not prime, then it is true. 
The function can make a mistake only is the case where a number $m$ is not prime, and is excidentally categorized by the function as prime. This can happen if all $100$ $a$'s that the function tried were not witnesses. According to the Miller-Rabin theorem $\frac{3}{4}$ of all possible $a$s are witnesses, so the probability for error is $(\frac{1}{4})^{100}$ (this is extremely low).

#### Testing the prime number theorem: For a large n, a number of n bits is prime with a prob. of O(1/n)
We decide on the size of the sample (to avoid testing all possible $2^{n-1}$ numbers of $n$ bits) and test whether each number we sample is prime. Then we divide the number of primes with the size of the sample. 

In [5]:
def prob_prime (n, sample):
    cnt = 0
    for i in range(sample):
        m = random.randint(2**(n-1), 2**n-1)
        cnt += is_prime(m)
    return cnt/sample

# prob_prime(2, 10**5)
prob_prime(200, 10**4)

0.0075

### Diffie Hellman from lecture

<img src="DH.PNG">

#### The protocol as code

In [6]:
def DH_exchange(p):
    """ generates a shared DH key """
    g=random.randint(1,p-1)
    a=random.randint(1,p-1)# Alice's  secret
    b=random.randint(1,p-1)# Bob's  secret
    x=pow(g,a,p)
    y=pow(g,b,p)
    key_A=pow(y,a,p)
    key_B=pow(x,b,p)
    #the next line is different from lecture
    return g, a, b, x, y, key_A #key_A=key_B

#### Find a prime number

In [7]:
def find_prime(n):
    """ find random n-bit long prime """
    while(True):
        candidate = random.randrange(2**(n-1),2**n)
        if is_prime(candidate):
            return candidate

Demostration:

In [8]:
import random
p = find_prime(10)
p
g,a,b,x,y,key = DH_exchange(p)
g,a,b,x,y,key

883

(196, 96, 702, 747, 46, 386)

In [9]:
pow(g, a, p)
pow(x, b, p)

747

386

#### Crack the Diffie Hellman code
There is no known way to find $a$ efficiently, so we try the naive one: iterating over all $a$-s and cheking whether the equation $g^a \mod p = x$ holds for them. 

If we found $a'$ that satisfies the condition but is not the original $a$, does it matter?

The time complexity of crack_DH is $O(2^nn^3)$

In [16]:
def crack_DH(p, g, x):
    ''' find secret "a" that satisfies g**a%p == x
        Not feasible for large p '''
    for a in range(1,p-1):
        if a%100000==0:
            print(a) #just to estimate running time
        if pow(g,a,p) == x:
            return a
    return None #should never get here

#### Trying to crack the protocol with a 100 bit prime

In [18]:
import random
p = find_prime(100)
# p
g,a,b,x,y,key = DH_exchange(p)
g,a,b,x,y,key

# crack_DH(p,g,x)

(585908982447390826849189842691,
 541120442351353498570334017705,
 1930761781644675344995819292,
 70659930174800164907189704268,
 779461075719798735415516947073,
 1045758344420625426709522664848)

Analyzing the nubmer of years it will take to crack the protocol if $a$ is found at the end (assuming iterating over 100000 $a$s takes a second)

In [20]:
541120442351353498570334017705//100000//60//60//24//365

171588166651240962

## OOP

A link to special method names in python: <a href="http://getpython3.com/diveintopython3/special-method-names.html">http://getpython3.com/diveintopython3/special-method-names.html</a>

### Basic polynomial class

We represent a polynomial as a list of coefficients, where the $i$'th element in the list is the coefficient of $x^i$

In [5]:
class Polynomial():
    def __init__(self, coeffs):
        self.coeffs = coeffs

    def __repr__(self):
        terms = [" + ("+str(self.coeffs[k])+"*x^" + \
                 str(k)+")" \
                 for k in range(len(self.coeffs)) \
                 if self.coeffs[k]!=0]
        terms = "".join(terms)
        if terms == "":
            return "0"
        else:
            return terms[3:] #discard leftmost '+'

f = Polynomial([1,1])
g = Polynomial([2,3,4])

f
g

(1*x^0) + (1*x^1)

(2*x^0) + (3*x^1) + (4*x^2)

#### Add degree and evaluate functions (these are special to class polynomial and are not part of Python's recognized functions)

In [6]:
class Polynomial():
    def __init__(self, coeffs):
        self.coeffs = coeffs

    def __repr__(self):
        terms = [" + ("+str(self.coeffs[k])+"*x^" + \
                 str(k)+")" \
                 for k in range(len(self.coeffs)) \
                 if self.coeffs[k]!=0]
        terms = "".join(terms)
        if terms == "":
            return "0"
        else:
            return terms[3:] #discard leftmost '+'

    def degree(self):
        return len(self.coeffs) - 1

    def evaluate(self, x):
        result = 0
        for i, c in enumerate(self.coeffs):
            result += c * pow(x, i)
        return result

f = Polynomial([1,1])
f.degree()
f.evaluate(5)

1

6

#### Add \__eq\__ function for the '==' operator
Without it, Python compares memory adderesses

In [7]:
class Polynomial():
    def __init__(self, coeffs):
        self.coeffs = coeffs

    def __repr__(self):
        terms = [" + ("+str(self.coeffs[k])+"*x^" + \
                 str(k)+")" \
                 for k in range(len(self.coeffs)) \
                 if self.coeffs[k]!=0]
        terms = "".join(terms)
        if terms == "":
            return "0"
        else:
            return terms[3:] #discard leftmost '+'

    def degree(self):
        return len(self.coeffs) - 1

    def evaluate(self, x):
        result = 0
        for i, c in enumerate(self.coeffs):
            result += c * pow(x, i)
        return result

    def __eq__(self, other):
        assert isinstance(other, Polynomial)  
        return self.coeffs == other.coeffs

f = Polynomial([1,1])
f2 = Polynomial([1,1])
g = Polynomial([2,3,4])

f == g
f == f2
f.__eq__(f2)

False

True

True

#### Add the \__add\__ function for the '+' operator

In [8]:
class Polynomial():
    def __init__(self, coeffs):
        self.coeffs = coeffs

    def __repr__(self):
        terms = [" + ("+str(self.coeffs[k])+"*x^" + \
                 str(k)+")" \
                 for k in range(len(self.coeffs)) \
                 if self.coeffs[k]!=0]
        terms = "".join(terms)
        if terms == "":
            return "0"
        else:
            return terms[3:] #discard leftmost '+'

    def degree(self):
        return len(self.coeffs) - 1

    def evaluate(self, x):
        result = 0
        for i, c in enumerate(self.coeffs):
            result += c * pow(x, i)
        return result

    def __eq__(self, other):
        assert isinstance(other, Polynomial)  
        return self.coeffs == other.coeffs

    def __add__(self, other):
        assert isinstance(other, (Polynomial, int))  
        if isinstance(other, int):
            terms = [c for c in self.coeffs]
            terms[0] += other
            return Polynomial(terms)
        #else, other is a polynomial
        size = min(self.degree(), other.degree()) + 1
        terms = [self.coeffs[i] + other.coeffs[i] for i in range(size)]
        terms += self.coeffs[size : self.degree() + 1]
        terms += other.coeffs[size : other.degree() + 1]
        #remove leading zeros
        last = len(terms)-1
        while last>0 and terms[last] == 0:
            last -= 1
        return Polynomial(terms[:last+1])

f = Polynomial([1,1])
g = Polynomial([2,3,4])

f+g

(3*x^0) + (4*x^1) + (4*x^2)

#### Add \__radd\__ function for right addition with int
Since there is no support for addition with a Polynomial in the class int, python needs a definition of right addition for int in class Polynomial

When executing the command $f + 3$, python will first search for an \__add\__ method in class int that can support a second operand of type polynomial (self = int, other = polynomial).
If such method does not exist, python will then search for an \__radd\__ method in class polynomial that can support a second operand of type int (self= polynomial, other = int).
If such method does not exist, an exception is thrown.

Since + is commutative, then we set \__radd\__ to be identical to \__add\__


In [9]:
class Polynomial():
    def __init__(self, coeffs):
        self.coeffs = coeffs

    def __repr__(self):
        terms = [" + ("+str(self.coeffs[k])+"*x^" + \
                 str(k)+")" \
                 for k in range(len(self.coeffs)) \
                 if self.coeffs[k]!=0]
        terms = "".join(terms)
        if terms == "":
            return "0"
        else:
            return terms[3:] #discard leftmost '+'

    def degree(self):
        return len(self.coeffs) - 1

    def evaluate(self, x):
        result = 0
        for i, c in enumerate(self.coeffs):
            result += c * pow(x, i)
        return result

    def __eq__(self, other):
        assert isinstance(other, Polynomial)  
        return self.coeffs == other.coeffs

    def __add__(self, other):
        assert isinstance(other, (Polynomial, int))  
        if isinstance(other, int):
            terms = [c for c in self.coeffs]
            terms[0] += other
            return Polynomial(terms)
        #else, other is a polynomial
        size = min(self.degree(), other.degree()) + 1
        terms = [self.coeffs[i] + other.coeffs[i] for i in range(size)]
        terms += self.coeffs[size : self.degree() + 1]
        terms += other.coeffs[size : other.degree() + 1]
        #remove leading zeros
        last = len(terms)-1
        while last>0 and terms[last] == 0:
            last -= 1
        return Polynomial(terms[:last+1])

    __radd__ = __add__ #to allow int+Polynomial

f = Polynomial([1,1])
1 + f

(2*x^0) + (1*x^1)

#### Add \__neg\__ function for unary minus (placing a '-' before a polynomial object)

In [37]:
class Polynomial():
    def __init__(self, coeffs):
        self.coeffs = coeffs

    def __repr__(self):
        terms = [" + ("+str(self.coeffs[k])+"*x^" + \
                 str(k)+")" \
                 for k in range(len(self.coeffs)) \
                 if self.coeffs[k]!=0]
        terms = "".join(terms)
        if terms == "":
            return "0"
        else:
            return terms[3:] #discard leftmost '+'

    def degree(self):
        return len(self.coeffs) - 1

    def evaluate(self, x):
        result = 0
        for i, c in enumerate(self.coeffs):
            result += c * pow(x, i)
        return result

    def __eq__(self, other):
        assert isinstance(other, Polynomial)  
        return self.coeffs == other.coeffs

    def __add__(self, other):
        assert isinstance(other, (Polynomial, int))  
        if isinstance(other, int):
            terms = [c for c in self.coeffs]
            terms[0] += other
            return Polynomial(terms)
        #else, other is a polynomial
        size = min(self.degree(), other.degree()) + 1
        terms = [self.coeffs[i] + other.coeffs[i] for i in range(size)]
        terms += self.coeffs[size : self.degree() + 1]
        terms += other.coeffs[size : other.degree() + 1]
        #remove leading zeros
        last = len(terms)-1
        while last>0 and terms[last] == 0:
            last -= 1
        return Polynomial(terms[:last+1])

    __radd__ = __add__ #to allow int+Polynomial
    
    def __neg__(self):
        return Polynomial([-co for co in self.coeffs])

f = Polynomial([1,1])
-f

(-1*x^0) + (-1*x^1)

#### Add \__sub\__  function to subtract one poly form another or an int from a poly
Using the fact we already have \__neg\__ and \__add\__ we define $f-g = f + (-g)$

In [11]:
class Polynomial():
    def __init__(self, coeffs):
        self.coeffs = coeffs

    def __repr__(self):
        terms = [" + ("+str(self.coeffs[k])+"*x^" + \
                 str(k)+")" \
                 for k in range(len(self.coeffs)) \
                 if self.coeffs[k]!=0]
        terms = "".join(terms)
        if terms == "":
            return "0"
        else:
            return terms[3:] #discard leftmost '+'

    def degree(self):
        return len(self.coeffs) - 1

    def evaluate(self, x):
        result = 0
        for i, c in enumerate(self.coeffs):
            result += c * pow(x, i)
        return result

    def __eq__(self, other):
        assert isinstance(other, Polynomial)  
        return self.coeffs == other.coeffs

    def __add__(self, other):
        assert isinstance(other, (Polynomial, int))  
        if isinstance(other, int):
            terms = [c for c in self.coeffs]
            terms[0] += other
            return Polynomial(terms)
        #else, other is a polynomial
        size = min(self.degree(), other.degree()) + 1
        terms = [self.coeffs[i] + other.coeffs[i] for i in range(size)]
        terms += self.coeffs[size : self.degree() + 1]
        terms += other.coeffs[size : other.degree() + 1]
        #remove leading zeros
        last = len(terms)-1
        while last>0 and terms[last] == 0:
            last -= 1
        return Polynomial(terms[:last+1])

    __radd__ = __add__ #to allow int+Polynomial
    
    def __neg__(self):
        return Polynomial([-co for co in self.coeffs])
    
    def __sub__(self, other):
        assert isinstance(other, (int,Polynomial))  
        return (self + (-other))
    
f = Polynomial([1,1])
g = Polynomial([2,3,4])

g - f
g - 1

(1*x^0) + (2*x^1) + (4*x^2)

(1*x^0) + (3*x^1) + (4*x^2)

#### Add \__rsub\__ function for right subtraction
Using the fact we already have \__neg\__ and \__sub\__ we define $f-g = -(g-f)$

since oprator \- is not commutative, then we cannot set \__rsub\__ to be identical to \__sub\__

Note that the following implementation of \__rsub\__ is promlematic, because it will keep calling itself (infinitely)

    def __rsub__(self, other):
        return other-self #PROBLEMATIC!




In [12]:
class Polynomial():
    def __init__(self, coeffs):
        self.coeffs = coeffs

    def __repr__(self):
        terms = [" + ("+str(self.coeffs[k])+"*x^" + \
                 str(k)+")" \
                 for k in range(len(self.coeffs)) \
                 if self.coeffs[k]!=0]
        terms = "".join(terms)
        if terms == "":
            return "0"
        else:
            return terms[3:] #discard leftmost '+'

    def degree(self):
        return len(self.coeffs) - 1

    def evaluate(self, x):
        result = 0
        for i, c in enumerate(self.coeffs):
            result += c * pow(x, i)
        return result

    def __eq__(self, other):
        assert isinstance(other, Polynomial)  
        return self.coeffs == other.coeffs

    def __add__(self, other):
        assert isinstance(other, (Polynomial, int))  
        if isinstance(other, int):
            terms = [c for c in self.coeffs]
            terms[0] += other
            return Polynomial(terms)
        #else, other is a polynomial
        size = min(self.degree(), other.degree()) + 1
        terms = [self.coeffs[i] + other.coeffs[i] for i in range(size)]
        terms += self.coeffs[size : self.degree() + 1]
        terms += other.coeffs[size : other.degree() + 1]
        #remove leading zeros
        last = len(terms)-1
        while last>0 and terms[last] == 0:
            last -= 1
        return Polynomial(terms[:last+1])

    __radd__ = __add__ #to allow int+Polynomial

    def __neg__(self):
        return Polynomial([-co for co in self.coeffs])

    def __sub__(self, other):
        assert isinstance(other, (int,Polynomial))  
        return (self + (-other))

    #__rsub__ = __sub__ #not what we want...

    def __rsub__(self, other):
        return -(self - other) #why not just other-self ?

f = Polynomial([1,1])
g = Polynomial([2,3,4])

1 - f
1 - g

(-1*x^1)

(-1*x^0) + (-3*x^1) + (-4*x^2)